In [15]:
import numpy as np
from scipy.optimize import minimize

# Parameters
n_individuals = 1000
mu_values = np.random.uniform(0.5, 2.0, size=n_individuals)
v_values = np.random.uniform(0.1, 1.0, size=n_individuals)

# Helper functions
def h(x): 
    return np.log(x + 1)  # Benefit from coverage

def psi(x):
    return np.log(x + 1)  # Risk adjustment

def k(x): 
    return 1.3*x  # Cost depending on coverage level

# Utility function
def utility(x, mu, v):
    return mu * h(x) + v * psi(x)

# Enrollment check
def is_enrolled(x, mu, v):
    return utility(x, mu, v) > utility(0, mu, v)  # Enroll if utility with coverage is greater than without

# Net value function
def net_value(x, mu, v):
    return utility(x, mu, v) - mu * k(x)

# Monopoly: Optimize for each individual
def optimize_monopoly():
    x_individual = np.zeros(n_individuals)

    for i in range(n_individuals):
        mu_i = mu_values[i]
        v_i = v_values[i]
        
        # Optimize individual net value given enrollment criteria
        res = minimize(lambda x: -net_value(x[0], mu_i, v_i) if is_enrolled(x[0], mu_i, v_i) else -net_value(0, mu_i, v_i),
                       [0.1], bounds=[(0, 1)])
        x_individual[i] = res.x[0] if is_enrolled(res.x[0], mu_i, v_i) else 0

    enrolled_mask = [is_enrolled(x, mu, v) for x, mu, v in zip(x_individual, mu_values, v_values)]

    number_enrolled_monopoly = np.sum(enrolled_mask)
    avg_cost_sharing_monopoly = np.mean([x for x, enrolled in zip(x_individual, enrolled_mask) if enrolled])
    avg_mu_monopoly = np.mean([mu for mu, enrolled in zip(mu_values, enrolled_mask) if enrolled])
    avg_risk_aversion_monopoly = np.mean([v for v, enrolled in zip(v_values, enrolled_mask) if enrolled])

    print("Monopoly Results:")
    print(f" - Number of Enrolled Individuals: {number_enrolled_monopoly}")
    print(f" - Avg Cost Sharing Among Enrolled: {avg_cost_sharing_monopoly:.4f}")
    print(f" - Avg μ Among Enrolled: {avg_mu_monopoly:.4f}")
    print(f" - Avg Risk Aversion Among Enrolled: {avg_risk_aversion_monopoly:.4f}")

# Competition: Optimize uniform \(x\) for average net value
def competitive_optimization():
    res = minimize(lambda x: -np.mean([net_value(x[0], mu, v) if is_enrolled(x[0], mu, v) else net_value(0, mu, v) for mu, v in zip(mu_values, v_values)]), 
                   [0.1], bounds=[(0, 1)])
    
    optimal_x_competitive = res.x[0]
    enrolled_mask_competitive = [is_enrolled(optimal_x_competitive, mu, v) for mu, v in zip(mu_values, v_values)]
    x_competitive = optimal_x_competitive if np.any(enrolled_mask_competitive) else 0

    number_enrolled_competitive = np.sum(enrolled_mask_competitive)
    avg_cost_sharing_competitive = x_competitive
    avg_mu_competitive = np.mean([mu for mu, enrolled in zip(mu_values, enrolled_mask_competitive) if enrolled])
    avg_risk_aversion_competitive = np.mean([v for v, enrolled in zip(v_values, enrolled_mask_competitive) if enrolled])

    print("Competition Results:")
    print(f" - Number of Enrolled Individuals: {number_enrolled_competitive}")
    print(f" - Avg Cost Sharing Among Enrolled: {avg_cost_sharing_competitive:.4f}")
    print(f" - Avg μ Among Enrolled: {avg_mu_competitive:.4f}")
    print(f" - Avg Risk Aversion Among Enrolled: {avg_risk_aversion_competitive:.4f}")

optimize_monopoly()
competitive_optimization()

Monopoly Results:
 - Number of Enrolled Individuals: 726
 - Avg Cost Sharing Among Enrolled: 0.2655
 - Avg μ Among Enrolled: 1.1670
 - Avg Risk Aversion Among Enrolled: 0.6801
Competition Results:
 - Number of Enrolled Individuals: 1000
 - Avg Cost Sharing Among Enrolled: 0.1221
 - Avg μ Among Enrolled: 1.2348
 - Avg Risk Aversion Among Enrolled: 0.5664
